In [1]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  


In [2]:
# MNIST Datensatz aus keras laden
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
# Das Shape des Datensatzes ausgeben
print("Ursprungs Shape des Trainingsdatensatzes:", x_train.shape)
print("Ursprungs Shape des Testdatensatzes:", x_test.shape)

Ursprungs Shape des Trainingsdatensatzes: (60000, 28, 28)
Ursprungs Shape des Testdatensatzes: (10000, 28, 28)


In [4]:
# Den Datensatz für ein SVM vorbereiten

# Flatten der Bildmatrizen für die SVM (von 28x28 zu 784)
x_train_svm = x_train.reshape((x_train.shape[0], -1))
x_test_svm = x_test.reshape((x_test.shape[0], -1))
# Standardisierung der Feature-Werte
scaler = StandardScaler()
x_train_svm = scaler.fit_transform(x_train_svm)
x_test_svm = scaler.transform(x_test_svm)

In [5]:
# Das Shape des Datensatzes ausgeben
print("SVM Shape des Trainingsdatensatzes:", x_train_svm.shape)
print("SVM Shape des Testdatensatzes:", x_test_svm.shape)

SVM Shape des Trainingsdatensatzes: (60000, 784)
SVM Shape des Testdatensatzes: (10000, 784)


In [11]:
# Parameteroptimierung mit GridSearchCV

# Parametergitter definieren
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 'scale', 0.0001],
    'kernel': ['rbf']
}

# GridSearchCV initialisieren
grid_search = GridSearchCV(svm.SVC(), param_grid, verbose=2, cv=3 , n_jobs=5, scoring='f1_weighted')

# Auf den Trainingsdaten fitten
grid_search.fit(x_train_svm, y_train)

# Beste Parameter-Kombination ausgeben
print("Beste Parameter-Kombination:", grid_search.best_params_)



Fitting 3 folds for each of 9 candidates, totalling 27 fits
Beste Parameter-Kombination: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [12]:
import pandas as pd

# Save cv_results_df to a CSV file
cv_results_df = pd.DataFrame(grid_search.cv_results_)
cv_results_df.to_csv('svm_v2_fmnist_gridsearch_f1.ipynb.csv', index=False)
print(cv_results_df)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0    5026.465348      6.492208      1868.205823        5.201673     0.1   
1    1337.842490     36.109089      1464.585855       89.880146     0.1   
2    1865.262038    172.896899      1885.408046      247.109314     0.1   
3    4196.918132     15.905177      1311.032965        1.905154       1   
4     697.431908    141.455508       785.040694       87.056561       1   
5     741.551321     61.083331      1046.950669       14.397416       1   
6    4107.253202     25.197321      1319.875941      296.727290      10   
7     735.258689     46.675170       910.584746      132.627745      10   
8     557.622598      5.689376       611.806159       71.072654      10   

  param_gamma param_kernel                                         params  \
0        0.01          rbf     {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}   
1       scale          rbf  {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}   
2      0.0001     